In [ ]:
import pandas as pd
from scipy.stats import zscore

In [ ]:
# Load the dataset
file_path = 'world-education-data.csv'  # Replace with your file path
data = pd.read_csv(file_path)
data.head()

,country,country_code,year,gov_exp_pct_gdp,lit_rate_adult_pct,pri_comp_rate_pct,pupil_teacher_primary,pupil_teacher_secondary,school_enrol_primary_pct,school_enrol_secondary_pct,school_enrol_tertiary_pct
0,Afghanistan,AFG,1999,NaN,NaN,NaN,33.18571,NaN,27.298849,NaN,NaN
1,Afghanistan,AFG,2000,NaN,NaN,NaN,NaN,NaN,22.162991,NaN,NaN
2,Afghanistan,AFG,2001,NaN,NaN,NaN,NaN,NaN,22.908590,14.47151,NaN
3,Afghanistan,AFG,2002,NaN,NaN,NaN,NaN,NaN,75.959747,NaN,NaN
4,Afghanistan,AFG,2003,NaN,NaN,NaN,NaN,NaN,96.553680,14.07805,1.38107


In [ ]:
# Step 1: Check for missing values
print("Missing values before cleaning:")
print(data.isnull().sum())


Missing values before cleaning:
country                          0
country_code                     0
year                             0
gov_exp_pct_gdp               1393
lit_rate_adult_pct            4015
pri_comp_rate_pct             1452
pupil_teacher_primary         2216
pupil_teacher_secondary       2875
school_enrol_primary_pct       540
school_enrol_secondary_pct    1147
school_enrol_tertiary_pct     1500
dtype: int64


In [ ]:
# Step 2: Handle missing values
# Drop columns with >50% missing values
columns_to_drop = ['lit_rate_adult_pct']  # Example: Replace with your column based on analysis
data_cleaned = data.drop(columns=columns_to_drop)


In [ ]:
# Impute missing values for numeric columns with the median
numeric_columns = data_cleaned.select_dtypes(include=['float64']).columns
data_cleaned[numeric_columns] = data_cleaned[numeric_columns].fillna(data_cleaned[numeric_columns].median())

In [ ]:
# Step 3: Remove outliers using z-scores (threshold > 3 standard deviations)
z_scores = data_cleaned[numeric_columns].apply(zscore)
outlier_mask = (z_scores.abs() > 3).any(axis=1)
data_cleaned_no_outliers = data_cleaned[~outlier_mask]


In [ ]:
# Step 4: Remove duplicates
data_cleaned_no_outliers = data_cleaned_no_outliers.drop_duplicates()

In [ ]:
# Display summary of the cleaned dataset
print("Missing values after cleaning:")
print(data_cleaned_no_outliers.isnull().sum())
print(f"Number of rows removed due to outliers: {data.shape[0] - data_cleaned_no_outliers.shape[0]}")
print(f"Final dataset shape: {data_cleaned_no_outliers.shape}")

Missing values after cleaning:
country                       0
country_code                  0
year                          0
gov_exp_pct_gdp               0
pri_comp_rate_pct             0
pupil_teacher_primary         0
pupil_teacher_secondary       0
school_enrol_primary_pct      0
school_enrol_secondary_pct    0
school_enrol_tertiary_pct     0
dtype: int64
Number of rows removed due to outliers: 465
Final dataset shape: (5427, 10)


In [ ]:
# Step 5: Feature Engineering
# Add a feature to indicate 20th or 21st century
data_cleaned_no_outliers['century'] = data_cleaned_no_outliers['year'].apply(lambda x: '20th' if x < 2000 else '21st')

In [ ]:
# Create a new feature for the difference between primary and secondary pupil-teacher ratios
data_cleaned_no_outliers['pupil_teacher_diff'] = data_cleaned_no_outliers['pupil_teacher_primary'] - data_cleaned_no_outliers['pupil_teacher_secondary']


In [ ]:
# Normalize enrollment percentages
enrollment_columns = ['school_enrol_primary_pct', 'school_enrol_secondary_pct', 'school_enrol_tertiary_pct']
data_cleaned_no_outliers[enrollment_columns] = data_cleaned_no_outliers[enrollment_columns].apply(lambda x: (x - x.min()) / (x.max() - x.min()))


In [ ]:
# Display a preview of the engineered dataset
print("Preview of the dataset after feature engineering:")
print(data_cleaned_no_outliers.head())

Preview of the dataset after feature engineering:
       country country_code  year  gov_exp_pct_gdp  pri_comp_rate_pct  \
3  Afghanistan          AFG  2002         4.101967          94.604504   
4  Afghanistan          AFG  2003         4.101967          94.604504   
5  Afghanistan          AFG  2004         4.101967          94.604504   
6  Afghanistan          AFG  2005         4.101967          94.604504   
7  Afghanistan          AFG  2006         4.684761          94.604504   

   pupil_teacher_primary  pupil_teacher_secondary  school_enrol_primary_pct  \
3              22.172125                 16.22447                  0.149829   
4              22.172125                 16.22447                  0.428541   
5              22.172125                 16.22447                  0.559804   
6              22.172125                 16.22447                  0.478291   
7              42.326020                 16.22447                  0.528400   

   school_enrol_secondary_pct  schoo

In [ ]:
# Step 6: Save the cleaned and engineered dataset
cleaned_file_path = 'cleaned_engineered_world_education_data.csv'  # Replace with desired file name
data_cleaned_no_outliers.to_csv(cleaned_file_path, index=False)
print(f"Cleaned and engineered dataset saved to {cleaned_file_path}")

Cleaned and engineered dataset saved to cleaned_engineered_world_education_data.csv


In [ ]:
df=pd.read_csv("cleaned_engineered_world_education_data.csv")
df.head()

,country,country_code,year,gov_exp_pct_gdp,pri_comp_rate_pct,pupil_teacher_primary,pupil_teacher_secondary,school_enrol_primary_pct,school_enrol_secondary_pct,school_enrol_tertiary_pct,century,pupil_teacher_diff
0,Afghanistan,AFG,2002,4.101967,94.604504,22.172125,16.22447,0.149829,0.512690,0.294246,21st,5.947655
1,Afghanistan,AFG,2003,4.101967,94.604504,22.172125,16.22447,0.428541,0.020558,0.012055,21st,5.947655
2,Afghanistan,AFG,2004,4.101967,94.604504,22.172125,16.22447,0.559804,0.056182,0.012137,21st,5.947655
3,Afghanistan,AFG,2005,4.101967,94.604504,22.172125,16.22447,0.478291,0.063388,0.294246,21st,5.947655
4,Afghanistan,AFG,2006,4.684761,94.604504,42.326020,16.22447,0.528400,0.127829,0.294246,21st,26.101550


In [ ]:
df['country']

,country
0,Afghanistan
1,Afghanistan
2,Afghanistan
3,Afghanistan
4,Afghanistan
...,...
5422,Zimbabwe
5423,Zimbabwe
5424,Zimbabwe
5425,Zimbabwe


In [ ]:
from geopy.geocoders import Nominatim
import pandas as pd

# List of countries
countries = df["country"]

# Initialize geolocator
geolocator = Nominatim(user_agent="geoapi")

# Get latitude and longitude
data = []
for country in countries:
    location = geolocator.geocode(country)
    if location:
        data.append({"country": country, "latitude": location.latitude, "longitude": location.longitude})
    else:
        data.append({"country": country, "latitude": None, "longitude": None})

# Convert to DataFrame
country_coords = pd.DataFrame(data)

# Save to CSV
country_coords.to_csv("country_coordinates.csv", index=False)
print(country_coords)


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Small+states&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))